In [1]:
from langchain.llms import Ollama
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever

from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

/home/joseacentic/centic/LLM/envPRUEBA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Para cargar los modelos en el servidor desdee hf: 
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/multilingual-e5-base')

In [3]:
'''
llm = Ollama(base_url="http://localhost:11434",
             model="llama3",
             verbose=True,  
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
'''
llm = Ollama(base_url="http://localhost:11434",
             model="mistral",
             verbose=True,  
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

hugg_embeddings = HuggingFaceEmbeddings(model_name= "mxbai-embed-large-v1")
#Este en concreto es en español
hugg_embeddings_jina = HuggingFaceEmbeddings(model_name= "jinaai/jina-embeddings-v2-base-es")

hugg_embeddings_e5 = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-base")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-es and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

In [18]:
import numpy as np
from transformers import AutoModel
from numpy.linalg import norm

sentence1 = ['hola muy buenos dias','Buenos dias como estas']

model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-es', trust_remote_code=True)


0.8340035

In [4]:
myFile = 'annualreport.pdf'
loader = PyPDFLoader("DATA/" + myFile)

data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len
    )

all_splits = text_splitter.split_documents(data)

In [5]:
#Los embeddings estan vacios!!
library = FAISS.from_documents(all_splits,hugg_embeddings_e5)
#library = FAISS.from_documents(all_splits,OllamaEmbeddings(model="mistral")

In [7]:
#Los scores devueltos son la distancia L2 que hay entre dos vectores
query = 'Who is Haroon Sheikh?'
library.similarity_search_by_vector(hugg_embeddings_e5.embed_query(query))

[Document(page_content='facilitates a culture of openness and debate.\nGroup Chief Executive Officer\nHaroon Sheikh is the Group CEO and \naccountable to the Board for the day-to-day \nrunning of the Group and management of  \nthe strategic plan.\nHaroon Sheikh is responsible for the following:\n –Executive leadership of the Group’s \nbusiness on a day-to-day basis.\n –Developing the overall commercial \nobjectives, and proposing and developing \nthe strategy in conjunction with the Board \nas a whole.', metadata={'source': 'DATA/annualreport.pdf', 'page': 42}),
 Document(page_content='in relation to his services as a Director on the Board. \nBenefits \nBenefits include car allowance (or company vehicle), vehicle expenses, healthcare insurance and relocation assistance. Relocation assistance \nwith a value of £144,000 was provided to Haroon Sheikh in the year, to assist with his move to the Middle East, and included visa and \nimmigration support, flights, housing and education fees.\n

In [77]:
library.similarity_search_by_vector(hugg_embeddings.embed_query(query))

[Document(page_content='sector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment', metadata={'source': 'DATA/annualreport.pdf', 'page': 40}),
 Document(page_content='facilitates a culture of openness and debate.\nGroup Chief Executive Officer\nHaroon Sheikh is the Group CEO and \naccountable to the Board for the day-to-day \nrunning of the Group and management of  \nthe strategic plan.\nHaroon Sheikh is responsible for the following:\n –Executive leadership of the Group’s \nbusiness on a day-to-day basis.\n –Developing the overall commercial \nobjectives, and proposing 

In [100]:
library.similarity_search_with_score("'Tell me all you know abour Karl Monaghan'")

[(Document(page_content='Cumming, Karl Monaghan and Professor \nMoira Livingston and are considered to be \nindependent. Although Karl Monaghan has \nserved on the Board for more than nine  \nyears, the Board are satisfied that there are  \nno matters which affects the independence  \nof his judgement and as such Karl continues  \nto act independently.The Board maintains regular focus on \nsuccession planning for both Board and \nsenior leadership roles. During the year, \nthe Board has considered the desired skills,', metadata={'source': 'DATA/annualreport.pdf', 'page': 42}),
  268.0751),
 (Document(page_content='accountant, having been admitted as a \nmember of the ICAEW in 2004 and as \na Fellow in 2014, and has a degree in \nComputer and Management Science \nfrom the University of Warwick.After graduating from University \nCollege Dublin with a Bachelor of \nCommerce degree, Karl trained as \na chartered accountant with KPMG \nin Dublin. He has worked in the \ncorporate finance dep

In [25]:
query_2 = 'How we can create value for our stakeholders?'
library.similarity_search_with_score(query_2)

[(Document(page_content='ensure we provide value in all areas of our \nbusiness. It is challenging to ensure all of our \nstakeholders have the same experience with \nthe Group, due to our wide range of locations, \noperations and roles; therefore, we promote an \nongoing dialogue with all our stakeholders to \nenable us to effectively act on feedback, and \nwe foster a culture of honesty and integrity.\nour approach to looking  \nafter people in our care\nWe believe the wellbeing of those entrusted', metadata={'source': 'DATA/annualreport.pdf', 'page': 24}),
  170.37415),
 (Document(page_content='12.75p for 2021.\nEngagement with our stakeholders\nThe six key stakeholders identified by the \nBoard are at the heart of what we do, being: \npeople in our care; our customers; our \nshareholders; our Regulators; people; and our \nsuppliers. It is of the highest importance to us \nthat we engage with all of our stakeholders meaningfully, to inform decision-making and \nensure we provide val

In [78]:
library.similarity_search_by_vector(hugg_embeddings.embed_query(query_2))

[Document(page_content='ensure we provide value in all areas of our \nbusiness. It is challenging to ensure all of our \nstakeholders have the same experience with \nthe Group, due to our wide range of locations, \noperations and roles; therefore, we promote an \nongoing dialogue with all our stakeholders to \nenable us to effectively act on feedback, and \nwe foster a culture of honesty and integrity.\nour approach to looking  \nafter people in our care\nWe believe the wellbeing of those entrusted', metadata={'source': 'DATA/annualreport.pdf', 'page': 24}),
 Document(page_content='12.75p for 2021.\nEngagement with our stakeholders\nThe six key stakeholders identified by the \nBoard are at the heart of what we do, being: \npeople in our care; our customers; our \nshareholders; our Regulators; people; and our \nsuppliers. It is of the highest importance to us \nthat we engage with all of our stakeholders meaningfully, to inform decision-making and \nensure we provide value in all areas 

In [ ]:
#Construir una cadena y pasarsela a un llm2 que será con llama 3 ok 
#hacer el prompt template y ver que contextos coge para hacer la respuesta. ok
#probar las diferentes distancias que vienen en la bdd de faiss y ver como funciona la distancia euclidea. ok solo tiene l2 y vector pero no deja elegirla en el retriver 
#comprobar tambien si se pueden hacer busquedas literales y no solo semánticas 
#ver como funciona el search with score (mas score mejor o menos) ok score es la distancia euclidea entre dos vectores
#controlar las alucinaciones ok

#ver como podemos seguir mejorando la primera parte por ejemplo chunks,, combinar dos llms o incluir algun 
#modelo de vision artificial... por ejemplo llavanext 

In [111]:
#query = who is Haaron Sheikh?
retriever_Ex = library.as_retriever(search_kwargs={"score_threshold": 200,"k": 4})
filtered_docs = retriever_Ex.get_relevant_documents('Tell me all you know abour Karl Monaghan')
filtered_docs


[]

In [8]:
#from langchain.chains import ConversationalRetrievalChain

#chain = ConversationalRetrievalChain.from_llm(llama3, library.as_retriever(), return_source_documents=True)

In [131]:
template = """
    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    It is mandatorian that if the answer is not in the context, answer "I have not enough context in order to answer this" and stop the answer.
"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)


In [132]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=library.as_retriever(search_kwargs={"score_threshold": 200,"k": 4}),
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

In [133]:
#pagina 07 imagen a la derecha
result = qa_chain({"query": "How many sites operate CareTech across the UK"})
result



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: CareTech has 16 years on the public markets 
and grown to over 550 services supporting 
close to 5,000 service users. The business has 
increased its range of services to offer a broad 
Care Pathway which was expanded during 
the year with the acquisition of Smartbox and 
the creation of a Digital Technology division. 
Continuing to be innovative and develop a 
range of Care Pathways will continue to help 
many of the adults, young people and children 
we support to live more independently.

management. We work closely with our 
regulators and commissioners across 
England, Scotland and Wales. 
CareTech is a well-known care Group in 
public ownership and offers high-quality 
services with a strong et

{'query': 'How many sites operate CareTech across the UK',
 'result': ' I have not enough context in order to answer this question as the number of sites operated by CareTech across the UK is not mentioned in the provided context.'}